## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Preambule](#prem)
     * [Package Loading](#package)
     * [Functions](#function)
* [LSTM](#lstm)
    * [1.FD001](#fd001)
        * [1.1 Data loading](#fd001dataload)
        * [1.2 Model selection](#fd001modelselect)
    * [2.FD002](#fd002)
         * [2.1 Data loading](#fd002dataload)
         * [2.2 Model selection](#fd002modelselect)
    * [3.FD003](#fd003)
         * [3.1 Data loading](#fd003dataload)
         * [3.2 Model selection](#fd003modelselect)
    * [4.FD004](#fd004)
         * [4.1 Data loading](#fd004dataload)
         * [4.2 Model selection](#fd004modelselect)

### Package loading <a class = "anchor" id="package"></a>

In [25]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt

from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
import keras
from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
import warnings

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
# import keras
import math

from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from methods import *
import warnings
from tensorflow.keras import optimizers
warnings.filterwarnings("ignore")
%matplotlib inline

warnings.filterwarnings('ignore')
print("okay")

okay


### Functions <a class = "anchor" id="function"></a>

In [49]:
# Fixer le seed pour la reproductibilité
SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [50]:
def rul_piecewise_fct(X_train, rul):
    
    X_train['RUL'].clip(upper=rul, inplace=True)
    
    return X_train

def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
    
    return X_train_interim, X_test_interim

# Models functions

# 1layers
def model_gru_1layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model
# 2layers
def model_gru_1layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

# 3layers
def model_gru_1layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

def model_gru_1layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

### FD001

In [14]:
# Data preparation
# Data loading

train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(20631, 27) (13096, 26) (100, 1)


In [51]:
rul_piecewise = 130
train['RUL'].clip(upper=rul_piecewise, inplace=True)

In [18]:
# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(10,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [[32], [64], [128], [256]]

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,3)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh']
batch_size_list = [32, 64, 128]
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options

5376

In [40]:
ITERATIONS = 1
SEED = 0

In [46]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
# parameter's sample
    weights_file = "model_weight/weights_file_gru.h5"
    alpha = 0.3
    sequence_length = 30
    epochs = 25
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_gru_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)

    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #       append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

iteration  1
(17731, 30, 14) (17731, 1) (100, 30, 14)
Epoch 1/25

278/278 [==============================] - 3s 6ms/step - loss: 6106.9746 - val_loss: 3876.1125
Epoch 2/25
278/278 [==============================] - 1s 5ms/step - loss: 4065.4397 - val_loss: 2679.3481
Epoch 3/25
278/278 [==============================] - 2s 5ms/step - loss: 2980.0967 - val_loss: 2048.1965
Epoch 4/25
278/278 [==============================] - 2s 5ms/step - loss: 2387.5076 - val_loss: 1761.6382
Epoch 5/25
278/278 [==============================] - 2s 5ms/step - loss: 2092.1121 - val_loss: 1666.6648
Epoch 6/25
278/278 [==============================] - 2s 5ms/step - loss: 1905.7988 - val_loss: 1386.4242
Epoch 7/25
278/278 [==============================] - 2s 6ms/step - loss: 1179.2554 - val_loss: 697.9668
Epoch 8/25
278/278 [==============================] - 2s 5ms/step - loss: 761.0727 - val_loss: 420.1109
Epoch 9/25
278/278 [==============================] - 2s 5ms/step - loss: 529.0710 - val_loss: 266.5

In [47]:
results

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
0,15.17286,0.0,472.867163,0.0,230.215729,0.0,[64],0.2,tanh,64
